# Домашняя работа

1. Масштабирование:
    а. Нормализовать признак - median_income
    б. Привести признак - population к масштабу от 0 до 100
    
2. Заполнение пропусков:
    а. Заполнить пропуски в столбце total_bedrooms средними значениями по категориям столбца ocean_proximity
    Например, если пропущенно значение в строке с категорией INLAND, нужно заполнить его средним значением по этой категории
    
3. Категорийные атрибуты:
    а. Посчитать среднее кол-во комнат в строке (total_rooms/households)
    б. Разбить полученный столбец на группы на ваше усмотрение (Например, 1 комната, 2 комнаты, 3 и более комнат)

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import scatter_matrix


%matplotlib inline

In [16]:
df = pd.read_csv('/Users/aleksandr/Desktop/1_лекция-Проблема_качества_данных/housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

**1. Масштабирование:** 
- а. Нормализовать признак - median_income
- б. Привести признак - population к масштабу от 0 до 100

In [17]:
df__x = df.copy()

x_array = np.array(df__x['median_income'])
normalized_X = preprocessing.normalize([x_array])

df__x['median_income_n'] = pd.DataFrame(normalized_X).T
df__x.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,median_income_n
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,0.013440
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,0.013401
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,0.011716


In [105]:
from sklearn.preprocessing import MinMaxScaler

df__y = df.copy()

scaler = MinMaxScaler()
df__y['population_m'] = np.round(scaler.fit_transform(df__y['population'].values.reshape(-1,1)) * 100, 2)
df__y.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,population_m
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,0.89
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.72
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,1.38


In [110]:
df__y.population_m.describe()

count    20640.000000
mean         3.986868
std          3.174039
min          0.000000
25%          2.200000
50%          3.260000
75%          4.830000
max        100.000000
Name: population_m, dtype: float64

**2. Заполнение пропусков:** 
- а. Заполнить пропуски в столбце total_bedrooms средними значениями по категориям столбца ocean_proximity 

Например, если пропущенно значение в строке с категорией INLAND, нужно заполнить его средним значением по этой категории

In [24]:
df__z = df.copy()

df_1 = df__z[df__z['ocean_proximity']=='NEAR BAY'].total_bedrooms.mean()
df_2 = df__z[df__z['ocean_proximity']=='NEAR OCEAN'].total_bedrooms.mean()
df_3 = df__z[df__z['ocean_proximity']=='ISLAND'].total_bedrooms.mean()
df_4 = df__z[df__z['ocean_proximity']=='INLAND'].total_bedrooms.mean()
df_5 = df__z[df__z['ocean_proximity']=='<1H OCEAN'].total_bedrooms.mean()

In [25]:
# df['total_bedrooms'] = df['total_bedrooms']fillna(value = df['total_bedrooms'].mean())
# df['total_bedrooms'] = df['total_bedrooms']fillna(value = -1)

df__z = df__z.replace(
    {
        'total_bedrooms': np.NaN
    },
    {
        'total_bedrooms': -1
    }
)

In [26]:
df__z['total_bedrooms'] = df__z.apply(lambda x: round(df_1, 0) if (x['ocean_proximity']=='NEAR BAY') & (x['total_bedrooms']==-1) 
                                 else (round(df_2, 0) if (x['ocean_proximity']=='NEAR OCEAN') & (x['total_bedrooms']==-1) 
                                       else(round(df_3, 0) if (x['ocean_proximity']=='ISLAND') & (x['total_bedrooms']==-1) 
                                            else(round(df_4, 0) if (x['ocean_proximity']=='INLAND') & (x['total_bedrooms']==-1) 
                                                 else(round(df_5, 0) if (x['ocean_proximity']=='<1H OCEAN') & (x['total_bedrooms']==-1) 
                                                      else x['total_bedrooms'])))), axis=1)

In [91]:
print('Среднее "NEAR BAY":', round(df_1, 0))
print('Среднее "NEAR OCEAN":', round(df_2, 0))
print('Среднее "ISLAND":', round(df_3, 0))
print('Среднее "INLAND":', round(df_4, 0))
print('Среднее "<1H OCEAN":', round(df_5, 0))

print('\n{0}\n\nИтого выбросов в данных: {1}'.format(df__z.isnull().sum(), df__z.isnull().sum().sum()))

Среднее "NEAR BAY": 514.0
Среднее "NEAR OCEAN": 539.0
Среднее "ISLAND": 420.0
Среднее "INLAND": 534.0
Среднее "<1H OCEAN": 547.0

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

Итого выбросов в данных: 0


In [29]:
# display(df__z.loc[(df__z['ocean_proximity'] == 'NEAR BAY') & (df__z['total_bedrooms'] == 514)].head(2))
# display(df__z.loc[(df__z['ocean_proximity'] == 'NEAR OCEAN') & (df__z['total_bedrooms'] == 539)].head(2))
# display(df__z.loc[(df__z['ocean_proximity'] == 'ISLAND') & (df__z['total_bedrooms'] == 420)].head(2)) #не было пустых ячеек
# display(df__z.loc[(df__z['ocean_proximity'] == 'INLAND') & (df__z['total_bedrooms'] == 534)].head(2))
# display(df__z.loc[(df__z['ocean_proximity'] == '<1H OCEAN') & (df__z['total_bedrooms'] == 547)].head(2))
# display(df__z.loc[(df__z['total_bedrooms'] == -1)].head())

**3. Категорийные атрибуты:**
- а. Посчитать среднее кол-во комнат в строке (total_rooms/households) 
- б. Разбить полученный столбец на группы на ваше усмотрение 

Например, 1 комната, 2 комнаты, 3 и более комнат

In [66]:
df__r = df.copy()

In [67]:
df__r['rooms_per_household'] = round(df__r['total_rooms']/df__r['households'], 0)

In [68]:
df__r.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,7.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.0


In [74]:
print('Min "rooms_per_household":', df__r.rooms_per_household.min())
print('Max "rooms_per_household":', df__r.rooms_per_household.max())

Min "rooms_per_household": 1.0
Max "rooms_per_household": 142.0


In [76]:
df__r['rooms_per_household_1'] = df__r.apply(lambda x: 1 if x['rooms_per_household']==1 else 0, axis =1)
df__r['rooms_per_household_2'] = df__r.apply(lambda x: 1 if x['rooms_per_household']==2 else 0, axis =1)
df__r['rooms_per_household_3'] = df__r.apply(lambda x: 1 if x['rooms_per_household']==3 else 0, axis =1)
df__r['rooms_per_household_4+'] = df__r.apply(lambda x: 1 if x['rooms_per_household']>=4 else 0, axis =1)

In [90]:
display(df__r.loc[(df__r['rooms_per_household_1'] == 1)].head(2))
display(df__r.loc[(df__r['rooms_per_household_2'] == 1)].head(2))
display(df__r.loc[(df__r['rooms_per_household_3'] == 1)].head(2))
display(df__r.loc[(df__r['rooms_per_household_4+'] == 1)].head(2))

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,rooms_per_household_1,rooms_per_household_2,rooms_per_household_3,rooms_per_household_4+
3126,-117.79,35.21,4.0,2.0,2.0,6.0,2.0,2.3750,137500.0,INLAND,1.0,1,0,0,0
4550,-118.23,34.05,52.0,346.0,270.0,346.0,251.0,2.5313,225000.0,<1H OCEAN,1.0,1,0,0,0


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,rooms_per_household_1,rooms_per_household_2,rooms_per_household_3,rooms_per_household_4+
50,-122.27,37.82,21.0,896.0,453.0,735.0,438.0,0.9218,171900.0,NEAR BAY,2.0,0,1,0,0
73,-122.29,37.81,46.0,12.0,4.0,18.0,7.0,0.4999,67500.0,NEAR BAY,2.0,0,1,0,0


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,rooms_per_household_1,rooms_per_household_2,rooms_per_household_3,rooms_per_household_4+
39,-122.26,37.83,52.0,2432.0,715.0,1377.0,696.0,2.5898,176000.0,NEAR BAY,3.0,0,0,1,0
49,-122.27,37.82,40.0,946.0,375.0,700.0,352.0,1.7750,112500.0,NEAR BAY,3.0,0,0,1,0


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,rooms_per_household_1,rooms_per_household_2,rooms_per_household_3,rooms_per_household_4+
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,7.0,0,0,0,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.0,0,0,0,1


In [ ]:
pass